In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
pip install PyMuPDF


In [3]:
import fitz  # PyMuPDF


In [4]:
!ollama pull llama3.2

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest 
pulling dde5aa3fc5ff... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 2.0 GB                         
pulling 966de95ca8a6... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.4 KB                         
pulling fcc5a6bec9da... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 7.7 KB                         
pulling a70ff7e570d9... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 6.0 KB                         
pulling 56bb8bd477a5... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ

In [5]:
import fitz  # PyMuPDF for PDF processing
import re
import requests
import gradio as gr
import time  # Measure time for performance testing

# Ollama API details
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"



In [6]:
# Initialize the OpenAI client for Ollama integration
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [7]:
!pip install pytesseract

In [8]:
import requests
import json
import gradio as gr
import fitz  # PyMuPDF for PDF text extraction
from openai import OpenAI

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

# Initialize the OpenAI client for Ollama integration
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

# Define the System message
system_message = "You are a highly knowledgeable assistant who can answer questions based on the user's resume. Use your understanding of the resume content to provide accurate and insightful answers."

# Function to extract text from the uploaded PDF resume (optimized)
def extract_text_from_pdf(pdf_file, max_pages=5):
    """
    Extracts text content from the uploaded PDF file. Optimized to extract from the first few pages.
    
    Args:
        pdf_file (file): The uploaded PDF resume file.
        max_pages (int): The number of pages to extract text from.
        
    Returns:
        str: Extracted text from the PDF.
    """
    try:
        # Open the PDF using PyMuPDF
        doc = fitz.open(pdf_file)
        text = ""
        
        # Limit the number of pages to extract from
        page_count = min(doc.page_count, max_pages)
        
        for page_num in range(page_count):
            page = doc.load_page(page_num)
            text += page.get_text("text")
        
        # Only return the text if there's something to extract
        if text.strip():
            return text.strip()
        else:
            return "No extractable text found in the PDF."

    except Exception as e:
        print(f"Error during PDF extraction: {e}")  # Log the error for debugging
        return f"Error extracting text from PDF: {e}"

# Function to answer questions based on the resume content
def answer_resume_question(question, resume_text, history):
    """
    Answers the user's question based on the resume content.
    
    Args:
        question (str): The user's question.
        resume_text (str): The extracted text from the resume.
        history (list): Conversation history.
        
    Returns:
        str: The answer based on the resume content.
        list: Updated conversation history.
    """
    try:
        # Create a prompt with the resume text and user's question
        prompt = (
            f"The following is a resume text:\n\n{resume_text}\n\n"
            f"Answer the following question based on the resume content:\n\n"
            f"Question: {question}\n\n"
            "Answer:"
        )
        
        # Make a request to the model
        response = ollama_via_openai.completions.create(
            model=MODEL,  # Llama model
            prompt=prompt,
            max_tokens=150,  # Limit the length of the answer
            temperature=0.6,  # Moderate creativity to avoid over-generation
        )
        
        # Extract the answer from the response
        answer = response.choices[0].text.strip()
        print(f"Answer from model: {answer}")  # Log the response for debugging
        
        # Append the question and answer to history
        history.append({"role": "user", "content": question})
        history.append({"role": "assistant", "content": answer})
        
        return answer, history

    except Exception as e:
        print(f"Error during model request: {e}")  # Log the error for debugging
        return f"An error occurred: {e}", history

# Define the Gradio UI and chat function
def chat_with_resume_qa(pdf_file, question, history):
    """
    Handles the user's question and answers based on the uploaded resume.
    
    Args:
        pdf_file (file): The uploaded resume PDF.
        question (str): The user's question about the resume.
        history (list): Conversation history.
        
    Returns:
        str: The assistant's answer based on the resume.
        list: Updated history.
    """
    # Ensure history is a list (if None, initialize as an empty list)
    history = history or []
    
    # Extract the text from the PDF
    resume_text = extract_text_from_pdf(pdf_file.name)
    
    if "Error" in resume_text or "No extractable text found" in resume_text:
        print(f"PDF extraction error: {resume_text}")  # Log extraction error
        return resume_text, history
    
    # Call the function to generate an answer based on the resume
    response, history = answer_resume_question(question, resume_text, history)
    
    return response, history

# Initialize the Gradio interface for resume-based Q&A
gr.Interface(
    fn=chat_with_resume_qa,
    inputs=[gr.File(label="Upload Your Resume (PDF)", type="filepath"),
            gr.Textbox(label="Ask a Question about the Resume", placeholder="Enter your question...", lines=2),
            gr.State()],
    outputs=[gr.Textbox(label="Assistant's Answer", interactive=False), gr.State()],
    live=False,  # Disable real-time submission
    title="Resume Q&A Chatbot",
    description="Upload your resume and ask questions about it. The chatbot will answer based on the resume content.",
    allow_flagging="never"
).launch(share = True)


C:\Users\GAD\anaconda3\envs\llms\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://67167cb9a1b4447f7e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Answer from model: Yes, this profile can develop high fidelity wireframes. The skills section of the resume mentions "Wireframing" as one of his techniques, and he has experience in developing low-fidelity wireframes during his UI/UX internship at CognoRise info tech. Additionally, he also mentions delivering interactive prototypes in his previous roles, which implies that he is familiar with creating detailed and realistic designs.

However, it's worth noting that the resume does not explicitly mention "High Fidelity Wireframing" as a specific skill or experience. But based on the information provided, it can be inferred that this profile has the capability to develop high fidelity wireframes, given his background in wireframing and prototyping.
